In [1]:
cd ~/workspace/

/home/itrushkin/workspace


In [2]:
# %env CUDA_VISIBLE_DEVICES=0 # choose a device
!fx collaborator start -n one

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-11-30 15:26:36.441613: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-30 15:26:36.441637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[15:26:37] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606739197.454369-209483;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606739197.4560685-356310;file:///home/itrushkin/.virtualenvs/fledge/li

[15:26:39] INFO     Histology > X_train Shape : (2000, 3, 150, 150)                                                                       ]8;id=1606739199.0633485-772700;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:128
           INFO     Histology > y_train Shape : (2000,)                                                                                   ]8;id=1606739199.0648267-450985;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:129
           INFO     Histology > Train Samples : 2000                                                                                      ]8;id=1606739199.066213-860231;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:130
           INFO     Histology > Valid Samples : 500                                                                                       ]8;id=1606739199.067493-731693;file:///home/itrushkin/workspace/code/histology_utils.

           INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606739201.1412585-394048;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:136
/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:234: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
[15:26:42] INFO     Sending metric for task aggregated_mode

[15:28:32] INFO     Sending metric for task locally_tuned_model_validation, round number 4: acc     0.63                                     ]8;id=1606739312.6284175-870743;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
[15:28:52] INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606739332.699718-543780;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:136
[15:28:57] INFO     Sending metric for task aggregated_model_validation, round number 5: acc        0.64                                     ]8;id=1606739337.7850947-366654;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
train epoch: 16it [00:04,  3.23it/s]
[15:29:

[15:30:50] INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606739450.678349-498162;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:136
[15:30:52] INFO     Sending metric for task aggregated_model_validation, round number 10: acc       0.734                                    ]8;id=1606739452.5789087-723387;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
train epoch: 16it [00:04,  3.21it/s]
[15:30:58] INFO     Sending metric for task train, round number 10: CrossEntropyLoss        0.7749706506729126                               ]8;id=1606739458.0241249-84975;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
[15:31:0

[15:32:54] INFO     Sending metric for task aggregated_model_validation, round number 15: acc       0.76                                     ]8;id=1606739574.4499338-240999;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
train epoch: 16it [00:05,  3.20it/s]
[15:32:59] INFO     Sending metric for task train, round number 15: CrossEntropyLoss        0.5855493545532227                               ]8;id=1606739579.906399-301910;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
[15:33:10] INFO     Sending metric for task locally_tuned_model_validation, round number 15: acc    0.774                                    ]8;id=1606739590.9224901-37175;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:318
[15:33:1